<a href="https://colab.research.google.com/github/sokhunter/face_mask_detection_app/blob/YOLO-test/Convertir_PASCAL_VOC_a_YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Crear nuevo Dataset listo para entrenamiento con YOLO

hecho por: Renzo Damián

Dataset utilizado: https://www.kaggle.com/datasets/andrewmvd/face-mask-detection

In [4]:
import os
import numpy as np
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Conectamos Google Drive y descrompimimos el dataset a convertir

In [ ]:
%cd drive/MyDrive/YOLO
%cd dataset/

[Errno 2] No such file or directory: 'drive/MyDrive/YOLO'
/content/drive/MyDrive/YOLO
/content/drive/MyDrive/YOLO/dataset


In [ ]:
!unzip archive.zip

Archive:  archive.zip
  inflating: annotations/maksssksksss0.xml  
  inflating: annotations/maksssksksss1.xml  
  inflating: annotations/maksssksksss10.xml  
  inflating: annotations/maksssksksss100.xml  
  inflating: annotations/maksssksksss101.xml  
  inflating: annotations/maksssksksss102.xml  
  inflating: annotations/maksssksksss103.xml  
  inflating: annotations/maksssksksss104.xml  
  inflating: annotations/maksssksksss105.xml  
  inflating: annotations/maksssksksss106.xml  
  inflating: annotations/maksssksksss107.xml  
  inflating: annotations/maksssksksss108.xml  
  inflating: annotations/maksssksksss109.xml  
  inflating: annotations/maksssksksss11.xml  
  inflating: annotations/maksssksksss110.xml  
  inflating: annotations/maksssksksss111.xml  
  inflating: annotations/maksssksksss112.xml  
  inflating: annotations/maksssksksss113.xml  
  inflating: annotations/maksssksksss114.xml  
  inflating: annotations/maksssksksss115.xml  
  inflating: annotations/maksssksksss116.xml

In [ ]:
%cd ..

/content/drive/MyDrive/YOLO


## Creamos la estructura para el dataset de entrenamiento

In [ ]:
classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']

In [ ]:
!mkdir "train_data"
!mkdir "train_data/images" 
!mkdir "train_data/labels" 
!mkdir "train_data/images/train" 
!mkdir "train_data/images/val" 
!mkdir "train_data/labels/train" 
!mkdir "train_data/labels/val" 

## Convertimos los archivos xml a txt y normalizamos las coordenadas

In [ ]:
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse

path = "./dataset/annotations"
classes = {"with_mask":0,"without_mask":1,"mask_weared_incorrect":2}

for annotations in os.listdir(path):
    dom = parse(os.path.join(path,annotations))
    root = dom.documentElement
    filename = ".txt".join(root.getElementsByTagName("filename")[0].childNodes[0].data.split(".png"))
    image_width = root.getElementsByTagName("width")[0].childNodes[0].data
    image_height = root.getElementsByTagName("height")[0].childNodes[0].data
    with open("./train_data/labels/train/"+filename,"w") as r:
        
        for items in root.getElementsByTagName("object") :
            
            name = items.getElementsByTagName("name")[0].childNodes[0].data
            xmin = items.getElementsByTagName("xmin")[0].childNodes[0].data
            ymin = items.getElementsByTagName("ymin")[0].childNodes[0].data
            xmax = items.getElementsByTagName("xmax")[0].childNodes[0].data
            ymax = items.getElementsByTagName("ymax")[0].childNodes[0].data
            x_center_norm = ((int(xmin)+int(xmax)) / 2 ) / int(image_width)
            y_center_norm = ((int(ymin)+int(ymax))/2) / int(image_height)
            width_norm = ((int(xmax)-int(xmin))/int(image_width))
            height_norm = ((int(ymax)-int(ymin))/int(image_height))
            
            r.write(str(classes[name]))
            r.write(" ")
            r.write(str(x_center_norm))
            r.write(" ")
            r.write(str(y_center_norm))
            r.write(" ")
            r.write(str(width_norm))
            r.write(" ")
            r.write(str(height_norm))
            r.write("\n")

## Copiar las imágenes del dataset a la carpeta train

In [7]:
import shutil
path = "./dataset/images"
for images in os.listdir(path):
    image_path = os.path.join(path,images)
    shutil.copy(image_path,"./train_data/images/train/"+images)

## Crear dataset de validación

In [8]:
from random import shuffle
from glob import glob
import random
import shutil

random.seed(42)
files = glob("./train_data/images/train/*.png")
shuffle(files)
txt_train_path = "./train_data/labels/train/"
img_train_path = "./train_data/images/train/"
txt_val_path = "./train_data/labels/val/"
img_val_path = "./train_data/images/val/"
for i in files[:120]:
    
    filename = i.split("/")[-1]
    shutil.move(txt_train_path+filename.replace(".png",".txt"),txt_val_path+filename.replace(".png",".txt"))
    shutil.move(img_train_path+filename,img_val_path+filename)


In [12]:
print(len(os.listdir("./train_data/labels/val/")) == len(os.listdir("./train_data/images/val/")),len(os.listdir("./train_data/labels/train/")) == len(os.listdir("./train_data/images/train/")))

True True


## Crear archivo de configuración

In [14]:
!mkdir data
!echo "train: /train_data/labels/train/" > data/facemask.yaml
!echo "val:   /train_data/images/val/" >> data/facemask.yaml
!echo "nc : 3" >> data/facemask.yaml
!echo "names: ['With_Mask', 'Without_Mask', 'Incorrect_Mask']" >> data/facemask.yaml

!cat data/facemask.yaml

train: /train_data/labels/train/
val:   /train_data/images/val/
nc : 3
names: ['With_Mask', 'Without_Mask', 'Incorrect_Mask']


## Listo! ya tenemos nuestro dataset configurado correctamente para entrenar YOLO